Import needed libraries

In [1]:
import pandas as pd

Read data

In [ ]:
# These links must not be changed and you can access data from them at anytime
# Just make sure to have quite stable internet connection
KAGGLE_DATA_PATH = 'hf://datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/kaggle_data.feather'
ZENODO_MENTALHEALTH_DATA_PATH = 'hf://datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/zenodo_mentalhealth_data.feather'
ZENODO_NOMENTALHEALTH_DATA_PATH = 'hf://datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/zenodo_nomentalhealth_data.feather'

In [3]:
kaggle_df = pd.read_feather(KAGGLE_DATA_PATH)

In [ ]:
kaggle_df.sample(n=20, random_state=123)

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0


In [5]:
zenodo_mentalhealth_df = pd.read_feather(ZENODO_MENTALHEALTH_DATA_PATH)

In [ ]:
zenodo_mentalhealth_df.sample(n=20, random_state=123)

,text,label
0,Hadn’t even made it a day Just relapsed for th...,addiction
1,I read my way out of addiction and into a mast...,addiction
2,I think my friend relapsed I haven't known her...,addiction


In [ ]:
zenodo_nomentalhealth_df = pd.read_feather(ZENODO_NOMENTALHEALTH_DATA_PATH)

In [ ]:
zenodo_nomentalhealth_df.sample(n=20, random_state=123)